<a href="https://colab.research.google.com/github/sourcecode369/deep-natural-language-processing/blob/master/language%20modelling/Learning_to_read_with_TensorFlow_and_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly
!pip install tensorflow-addons
!pip install keras-tuner

     |████████████████████████████████| 531.4MB 32kB/s 
     |████████████████████████████████| 2.8MB 54.3MB/s 
     |████████████████████████████████| 2.9MB 53.0MB/s 
     |████████████████████████████████| 460kB 51.1MB/s 
     |████████████████████████████████| 778kB 52.2MB/s 
ERROR: tensorflow 1.15.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
  Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Found existing installation: h5py 2.8.0
    Uninstalling h5py-2.8.0:
      Successfully uninstalled h5py-2.8.0
     |████████████████████████████████| 1.0MB 40.3MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=31b6800c7a477069a8c53ae80a9ef290f3b148f67a80bad5f4f23b919ca5c941
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wh

In [2]:
import tensorflow as tf
import tensorflow_addons as tfa
print(tf.__version__)
dir(tfa.seq2seq)

2.2.0-dev20200313


['AttentionMechanism',
 'AttentionWrapper',
 'AttentionWrapperState',
 'BahdanauAttention',
 'BahdanauMonotonicAttention',
 'BaseDecoder',
 'BasicDecoder',
 'BasicDecoderOutput',
 'BeamSearchDecoder',
 'BeamSearchDecoderOutput',
 'BeamSearchDecoderState',
 'CustomSampler',
 'Decoder',
 'FinalBeamSearchDecoderOutput',
 'GreedyEmbeddingSampler',
 'InferenceSampler',
 'LuongAttention',
 'LuongMonotonicAttention',
 'SampleEmbeddingSampler',
 'Sampler',
 'ScheduledEmbeddingTrainingSampler',
 'ScheduledOutputTrainingSampler',
 'SequenceLoss',
 'TrainingSampler',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'attention_wrapper',
 'basic_decoder',
 'beam_search_decoder',
 'decoder',
 'dynamic_decode',
 'gather_tree',
 'gather_tree_from_array',
 'hardmax',
 'loss',
 'monotonic_attention',
 'safe_cumprod',
 'sampler',
 'sequence_loss',
 'tile_batch']

In [3]:
!wget http://www.thespermwhale.com/jaseweston/babi/CBTest.tgz
!tar -xf CBTest.tgz

--2020-03-13 12:01:46--  http://www.thespermwhale.com/jaseweston/babi/CBTest.tgz
Resolving www.thespermwhale.com (www.thespermwhale.com)... 69.65.3.213
Connecting to www.thespermwhale.com (www.thespermwhale.com)|69.65.3.213|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120547669 (115M) [application/x-tar]
Saving to: ‘CBTest.tgz’

CBTest.tgz          100%[===================>] 114.96M  11.2MB/s    in 12s     

2020-03-13 12:01:58 (9.92 MB/s) - ‘CBTest.tgz’ saved [120547669/120547669]

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tar: Ignoring unknown extended header keyword 'SCHILY.

In [4]:
!ls CBTest/data

cbtest_CN_test_2500ex.txt   cbtest_P_valid_2000ex.txt
cbtest_CN_train.txt	    cbtest_V_test_2500ex.txt
cbtest_CN_valid_2000ex.txt  cbtest_V_train.txt
cbtest_NE_test_2500ex.txt   cbtest_V_valid_2000ex.txt
cbtest_NE_train.txt	    cbt_test.txt
cbtest_NE_valid_2000ex.txt  cbt_train.txt
cbtest_P_test_2500ex.txt    cbt_valid.txt
cbtest_P_train.txt


In [5]:
lines = tf.data.TextLineDataset('CBTest/data/cbt_train.txt')
for row in lines.take(3):
  print(row)

tf.Tensor(b'_BOOK_TITLE_ : Andrew_Lang___Prince_Prigio.txt.out', shape=(), dtype=string)
tf.Tensor(b'CHAPTER I. -LCB- Chapter heading picture : p1.jpg -RCB- How the Fairies were not Invited to Court .', shape=(), dtype=string)
tf.Tensor(b'Once upon a time there reigned in Pantouflia a king and a queen .', shape=(), dtype=string)


In [7]:
lines  = lines.filter(
    lambda x: not tf.strings.regex_full_match(x, "_BOOK_TITLE_.*")
)

punctuation = r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']'

lines = lines.map(lambda x: tf.strings.regex_replace(x, punctuation, ' '))

for row in lines.take(3):
  print(row)

tf.Tensor(b'CHAPTER I   LCB  Chapter heading picture   p1 jpg  RCB  How the Fairies were not Invited to Court  ', shape=(), dtype=string)
tf.Tensor(b'Once upon a time there reigned in Pantouflia a king and a queen  ', shape=(), dtype=string)
tf.Tensor(b'With almost everything else to make them happy   they wanted one thing   they had no children  ', shape=(), dtype=string)


In [8]:
words = lines.map(tf.strings.split)
wordsets = words.unbatch().batch(11)
for row in wordsets.take(3):
  print(row)

tf.Tensor(
[b'CHAPTER' b'I' b'LCB' b'Chapter' b'heading' b'picture' b'p1' b'jpg'
 b'RCB' b'How' b'the'], shape=(11,), dtype=string)
tf.Tensor(
[b'Fairies' b'were' b'not' b'Invited' b'to' b'Court' b'Once' b'upon' b'a'
 b'time' b'there'], shape=(11,), dtype=string)
tf.Tensor(
[b'reigned' b'in' b'Pantouflia' b'a' b'king' b'and' b'a' b'queen' b'With'
 b'almost' b'everything'], shape=(11,), dtype=string)


In [10]:
def get_example_label(row):
  example = tf.strings.reduce_join(row[:-1], separator = ' ')
  example = tf.expand_dims(example, axis=0)
  label = row[-1:]
  return example, label

data = wordsets.map(get_example_label)
data = data.shuffle(1000)
for row in data.take(3):
  print(row)

(<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'hoping that either the Firedrake would roast Prince Prigio alive'],
      dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'LRB'], dtype=object)>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'and when the queen looked up lo and behold on'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'every'], dtype=object)>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'of our aunts No the old cats replied the queen'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'for'], dtype=object)>)


In [0]:
max_features = 10000
vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=max_features,
    output_sequence_length=10
)

vectorize_layer.adapt(lines.batch(64))

In [13]:
vectorize_layer.get_vocabulary()[:5]

[b'the', b'and', b'to', b'a', b'of']

In [14]:
vectorize_layer.get_vocabulary()[-5:]

[b'custard', b'curtained', b'cupboards', b'culture', b'cuckoo']

In [15]:
for batch in data.batch(3).take(1):
  print(batch[0])
  print(vectorize_layer(batch[0]))

tf.Tensor(
[[b'determined to challenge the gentleman who was in the carriage']
 [b'in the valley where it was not so very cold']
 [b'bottom of the hill and shouted Hi Well grunted the']], shape=(3, 1), dtype=string)
tf.Tensor(
[[1288    4 7937    2  789   60    9   11    2 1693]
 [  11    2  884   95   10    9   29   32   56  423]
 [1083    6    2  451    3 1006 3701   85 5301    2]], shape=(3, 10), dtype=int64)


In [0]:
class EncoderDecoder(tf.keras.Model):
  def __init__(self, max_features=5000, embedding_dims = 200, rnn_units=1024):
    super().__init__()
    self.max_features = max_features
    
    self.vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
        max_tokens=max_features,
        output_sequence_length=10
    )
    
    self.encoder_embedding = tf.keras.layers.Embedding(max_features+1, embedding_dims)
    self.lstm_layer = tf.keras.layers.LSTM(rnn_units, return_state=True)

    self.decoder_embedding = tf.keras.layers.Embedding(max_features+1, embedding_dims)
    sampler = tfa.seq2seq.sampler.TrainingSampler()
    decoder_cell = tf.keras.layers.LSTMCell(rnn_units)
    projection_layer = tf.keras.layers.Dense(max_features)
    self.decoder = tfa.seq2seq.BasicDecoder(decoder_cell, sampler, projection_layer)
    self.attention = tf.keras.layers.Attention()

  def train_step(self, data):
    x, y = data[0], data[1]
    x = self.vectorize_layer(x)
    y = self.vectorize_layer(y)[:, 0:1]
    y_one_hot = tf.one_hot(y, self.max_features)
    with tf.GradientTape() as tape:
      embedded_inputs = self.encoder_embedding(x)
      encoder_outputs, state_h, state_c = self.lstm_layer(embedded_inputs)
      attn_output = self.attention([encoder_outputs, state_h])
      attn_output = tf.expand_dims(attn_output, axis=1)
      targets = self.decoder_embedding(tf.zeros_like(y))
      concat_output = tf.concat([targets, attn_output], axis=-1)
      outputs, _, _ = self.decoder(concat_output, initial_state=[state_h, state_c])

      y_pred = outputs.rnn_output

      loss = self.compiled_loss(
          y_one_hot, 
          y_pred,
          regularization_losses = self.losses
      )
      trainable_variables = self.trainable_variables
      gradients = tape.gradient(loss, trainable_variables)
      self.optimizer.apply_gradients(zip(gradients, trainable_variables))
      self.compiled_metrics.update_state(y_one_hot, y_pred)
      return {m.name:m.result() for m in self.metrics}
    
  def predict_step(self, data, select_from_top_n=1):
    x = data
    if isinstance(x, tuple) and len(x) == 2:
      x = x[0]
    x = self.vectorize_layer(x)
    embedded_inputs = self.encoder_embedding(x)
    encoder_outputs, state_h, state_c = self.lstm_layer(embedded_inputs)
    attn_output = self.attention([encoder_outputs, state_h])
    attn_output = tf.expand_dims(attn_output, axis=1)
    
    targets = self.decoder_embedding(tf.zeros_like(x[:, -1:]))
    concat_output = tf.concat([targets, attn_output], axis=-1)
    outputs, _, _ = self.decoder(
        concat_output, initial_state=[state_h, state_c])
    
    y_pred = tf.squeeze(outputs.rnn_output, axis=1)
    top_n = tf.argsort(
        y_pred[:, 2:], axis=1, direction='DESCENDING')[: ,:select_from_top_n]
    chosen_indices = tf.random.uniform(
        [top_n.shape[0], 1], minval=0, maxval=select_from_top_n, 
        dtype=tf.dtypes.int32)
    counter = tf.expand_dims(tf.range(0, top_n.shape[0]), axis=1)
    indices = tf.concat([counter, chosen_indices], axis=1)
    choices = tf.gather_nd(top_n, indices)
    words = [self.vectorize_layer.get_vocabulary()[i] for i in choices]
    return words

  def predict(self, starting_string, num_steps=50, select_from_top_n=1):
    s = tf.compat.as_bytes(starting_string).split(b' ')
    for _ in range(num_steps):
      windowed = [b' '.join(s[-10:])]
      pred = self.predict_step([windowed], select_from_top_n=select_from_top_n)
      s.append(pred[0])
    return b' '.join(s)

In [0]:
model = EncoderDecoder()
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
    optimizer='adam', 
    metrics=['accuracy'])
model.vectorize_layer.adapt(lines.batch(256))

In [18]:
model.fit(data.batch(256), epochs=30, callbacks=[tf.keras.callbacks.ModelCheckpoint('text_gen_ckpt')])

Epoch 1/30
1690/1690 [==============================] - 122s 72ms/step - loss: 5.4021 - accuracy: 0.1295
Epoch 2/30
1690/1690 [==============================] - 125s 74ms/step - loss: 4.7577 - accuracy: 0.1699
Epoch 3/30
1690/1690 [==============================] - 124s 74ms/step - loss: 4.3987 - accuracy: 0.1892
Epoch 4/30
1690/1690 [==============================] - 124s 74ms/step - loss: 3.9583 - accuracy: 0.2188
Epoch 5/30
1690/1690 [==============================] - 124s 74ms/step - loss: 3.3707 - accuracy: 0.2809
Epoch 6/30
1690/1690 [==============================] - 124s 74ms/step - loss: 2.7337 - accuracy: 0.3831
Epoch 7/30
1690/1690 [==============================] - 124s 74ms/step - loss: 2.1562 - accuracy: 0.4970
Epoch 8/30
1690/1690 [==============================] - 125s 74ms/step - loss: 1.6845 - accuracy: 0.5996
Epoch 9/30
1690/1690 [==============================] - 125s 74ms/step - loss: 1.3354 - accuracy: 0.6769
Epoch 10/30
1690/1690 [==============================] 

KeyboardInterrupt: ignored

In [19]:
model.fit(data.batch(256), epochs=10, callbacks=[tf.keras.callbacks.ModelCheckpoint('text_gen_ckpt')])

Epoch 1/10
    285/Unknown - 20s 71ms/step - loss: 1.0169 - accuracy: 0.7432

KeyboardInterrupt: ignored

In [20]:
model.load_weights('text_gen_ckpt')

In [21]:
print(model.predict('The mouse and the rabbit went in together'))

print(model.predict('Once upon a time there was a Queen named Darling'))

print(model.predict('In a city far from here the teacup shook upon the table'))

print(model.predict('It was a strange and quiet theater and the people watched from home'))

b'The mouse and the rabbit went in together and and from the first time he felt that he could n t see any one in the green forest indeed what is it asked sammy pretending to look out of the doorway yes sir it was he he cried it is all the best of you know tell these'
b'Once upon a time there was a Queen named Darling alive in a certain voice he learned it out before it with his sharp teeth and with a little noise of tail the water was soft and this was her long cry grandfather frog had grown there no longer there was such a thing as mrs quack came into the'
b'In a city far from here the teacup shook upon the table which were on their side it had been the most beautiful old appetite but farmer brown s boy would n t eat him until he was a prisoner or two not at all i do n t think so i can do just what i do n t know who'
b'It was a strange and quiet theater and the people watched from home toward last was it the very best thing to be is to know what was going on about him then peter rabbi

In [0]:
import kerastuner as kt

def build_model(hp):
  model = EncoderDecoder(
      rnn_units=hp.Int('units', min_value=256, max_value=1200, step=256))
  
  model.compile(
      optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-3, 1e-4, 3e-4])),
      loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),                 
      metrics=['accuracy'])
  
  model.vectorize_layer.adapt(lines.batch(256))
  return model

tuner = kt.tuners.RandomSearch(
    build_model,
    objective='accuracy',
    max_trials=15,
    executions_per_trial=1,
    directory='my_dir',
    project_name='text_generation')

tuner.search(
    data.batch(256), 
    epochs=10, 
    callbacks=[tf.keras.callbacks.ModelCheckpoint('text_gen_ckpt')])

Epoch 1/10
    718/Unknown - 55s 76ms/step - loss: 6.1330 - accuracy: 0.0669